In [1]:
# N5 (Ensemble) : Food-256 | Food-101 | Rice dataset

In [2]:
## imports
import os
import numpy as np
import torch
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import torch.nn as nn
%matplotlib inline

In [3]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()
device = 'cpu'
if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
    device = 'cuda'

CUDA is available!  Training on GPU ...


In [4]:
## function to train the network
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    train_loss = 0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        
    train_loss /= len(train_loader.dataset)
    print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch, correct, len(train_loader.dataset),
                100. * correct / len(train_loader.dataset), train_loss))

## function to train the network
def test(model, device, criterion, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test loss: {:.4f}, Test Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [5]:
data_transform =  transforms.Compose([
    transforms.Resize(250),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [6]:
def loadN(path,n_classes):
    
    """
    Function to load VGG pretrained models
    """
    model = models.vgg19(pretrained=False)
    n_inputs = model.classifier[-1].in_features
    last_layer = nn.Linear(n_inputs,n_classes)
    model.classifier[-1] = last_layer
    model.load_state_dict(torch.load(path, map_location=device), strict=False)
    # if GPU is available, move the model to GPU
    if train_on_gpu:
        model.cuda()
    # check to see that your last layer produces the expected number of outputs
    return model

In [7]:
food101trained = loadN('food-101-trained.pwf',101)
food101trained.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace)
  (2): Dropout(p=0.5)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace)
  (5): Dropout(p=0.5)
  (6): Linear(in_features=4096, out_features=101, bias=True)
)

In [8]:
food256trained = loadN('food-256-trained.pwf',256)
food256trained.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace)
  (2): Dropout(p=0.5)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace)
  (5): Dropout(p=0.5)
  (6): Linear(in_features=4096, out_features=256, bias=True)
)

In [9]:
#vgg = models.vgg19(pretrained=False)
#vgg

In [10]:
class N5(nn.Module):
    def __init__(self, modelA, modelB):
        super(N5, self).__init__()
        
        self.modelA = modelA
        for param in self.modelA:
            param.requires_grad = False
            
        self.modelB = modelB
        for param in self.modelB:
            param.requires_grad = False
            
        self.classifier = nn.Sequential(
            nn.Linear(50176,4096),
            nn.ReLU(),
            nn.Linear(4096,4)
        )
        
    def flatten(self, dim, data):
        data = data.transpose(dim, -1).contiguous()
        return data.view(-1, data.size(-1)).t()
    
    def forward(self, x):
        xA = self.modelA(x)
        xA = self.flatten(0, xA)
        xB = self.modelA(x)
        xB = self.flatten(0, xB)
        
        x_all = torch.cat([xA, xB], dim=1)   
        
        pred = self.classifier(nn.functional.relu(x_all))
        
        return pred
        

In [11]:
model = N5(food101trained.features, food256trained.features)
if train_on_gpu:
    model.cuda()

In [12]:

data_dir = 'D://Datasets//RICE_DATASET//'
batch_size = 1
num_workers = 0


train_data = datasets.ImageFolder(
        data_dir+'training_data', 
        transform=data_transform)

test_data = datasets.ImageFolder(
        data_dir+'test_data', 
        transform=data_transform)


print(len(train_data)+len(test_data),"images")
# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True)


428 images


In [13]:
import torch.optim as optim
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
# specify optimizer and learning rate
optimizer = optim.Adam(model.classifier.parameters(), lr=0.0001)

In [14]:
# number of epochs to train the model
n_epochs = 30

print ("Start training")

for epoch in range(1, n_epochs+1):
    
    train (model, device, train_loader, optimizer, criterion, epoch)
    test (model, device, criterion, test_loader)
    
    print ('\n')
            
print ("Done training")

Start training
Epoch: 1 , Training Accuracy: 269/348 (77%) Training Loss: 1.415750
Test loss: 1.1222, Test Accuracy: 59/80 (74%)



Epoch: 2 , Training Accuracy: 323/348 (93%) Training Loss: 0.283595
Test loss: 1.3066, Test Accuracy: 60/80 (75%)



Epoch: 3 , Training Accuracy: 347/348 (100%) Training Loss: 0.029455
Test loss: 0.7729, Test Accuracy: 64/80 (80%)



Epoch: 4 , Training Accuracy: 348/348 (100%) Training Loss: 0.000144
Test loss: 0.7892, Test Accuracy: 64/80 (80%)



Epoch: 5 , Training Accuracy: 348/348 (100%) Training Loss: 0.000053
Test loss: 0.7493, Test Accuracy: 64/80 (80%)



Epoch: 6 , Training Accuracy: 348/348 (100%) Training Loss: 0.000028
Test loss: 0.7474, Test Accuracy: 64/80 (80%)



Epoch: 7 , Training Accuracy: 348/348 (100%) Training Loss: 0.000015
Test loss: 0.6916, Test Accuracy: 66/80 (82%)



Epoch: 8 , Training Accuracy: 348/348 (100%) Training Loss: 0.000009
Test loss: 0.7060, Test Accuracy: 64/80 (80%)



Epoch: 9 , Training Accuracy: 348/348 (100%

In [15]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        if train_on_gpu:
            images, labels = images.cuda(), labels.cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 80 %


In [16]:
torch.save(model.state_dict(), 'newN5.pwf')